<p><img alt="udeA logo" height="150px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true" align="left" hspace="50px" vspace="0px" style="width:107px;height:152px;"></p>
<h1><font color='#FFFFFFF'> <center>
Proyecto 3: Filtros</center></font></h1>
<h2><font color='#FFFFFFF'> <center>
Proyecto 2024-02 </center></font></h2>
<h3><font color='#FFFFFFF'> <center>María J. Ostos - Cristian Florez - Juan A. Sañudo</center></font></h3>
<h3><font color='#FFFFFFF'> <center>
2024 </center></font></h3>
<font  face="Courier New" size="3">
<p1><center> </center></p1>

1. Del artículo: https://www.nature.com/articles/s41598-020-59821-7

Consultar y explicar los dos métodos de reducción de ruido usados en el artículo: Robust LOESS y Non Local Means (10%), mostrar cómo se podrían implementar en Python (5%)

2. Descargar los datos ECGData.zip de:
https://figshare.com/collections/ChapmanECG/4560497/1

De la base de datos extraer los registros que correspondan a bradicardia sinusal (SB Sinus Bradycardia) y fibrilación auricular (AFIB Atrial Fibrillation). Esta información está en el archivo Diagnostics.xlsx

### Carga de librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fftpack import fft, ifft, fftfreq
from scipy.signal import detrend
from scipy.signal import welch
from numpy.polynomial.polynomial import Polynomial
import os
import pywt
from scipy import stats
import random

### Carga de datos Diagnostics.xlsx

In [3]:
fs=500

ruta_diagnostics="./datos/Diagnostics.xlsx"
df_diagnostics = pd.read_excel(ruta_diagnostics)

# Crear el nuevo dataframe filtrando por la columna 'Rhythm'
datos_diagnostics = df_diagnostics[(df_diagnostics['Rhythm'] == 'SB') | (df_diagnostics['Rhythm'] == 'AFIB')]

#Tamaño del nuevo dataframe
datos_diagnostics.shape

tiempo_diagnostics = np.arange(0, datos_diagnostics.shape[0]/fs,1/fs)

datos_diagnostics.head()

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,RAxis,TAxis,QRSCount,QOnset,QOffset,TOffset
0,MUSE_20180113_171327_27000,AFIB,RBBB TWC,85,MALE,117,234,114,356,496,81,-27,19,208,265,386
1,MUSE_20180112_073319_29000,SB,TWC,59,FEMALE,52,52,92,432,401,76,42,8,215,261,431
3,MUSE_20180113_121940_44000,SB,NONE,66,MALE,53,53,96,456,427,34,3,9,219,267,447
5,MUSE_20180112_120347_79000,SB,NONE,46,FEMALE,57,57,70,404,393,38,24,9,225,260,427
6,MUSE_20180114_075026_69000,AFIB,TWC,80,FEMALE,98,86,74,360,459,69,83,17,215,252,395


3. Consultar que otros tipos de señales wavelet se pueden usar para el análisis de señales ECG y adaptar el código del filtro wavelet que se entrega en el curso de acuerdo a la consulta (10%)

### Filtro Wavelet dado en el curso

In [3]:
def wnoisest(coeff):
    stdc = np.zeros((len(coeff),1));
    for i in range(0,len(coeff)):
        stdc[i] = (np.median(np.absolute(coeff[i])))/0.6745;
    return stdc;

In [4]:
def threshold(coeff):
    Num_samples = 0;
    for i in range(0,len(coeff)):
        Num_samples = Num_samples + coeff[i].shape[0];
    
    thr = np.sqrt(2*(np.log(Num_samples)))
    return thr

In [5]:
def wthresh(coeff):
    y   = list();
    s = wnoisest(coeff);
    print(s)
    thr = threshold(coeff)
    print(thr)
    for i in range(0,len(coeff)):
        y.append(np.multiply(coeff[i],np.abs(coeff[i])>(thr*s[i])));
    return thr,s,y;

In [ ]:

def filt_wav(dato):
    # data=datos_II[dato]
    data=dato
    LL = int(np.floor(np.log2(data.shape[0])));

    data_wavelet = pywt.wavedec( data, 'db6', level=8 );
    print(len(data_wavelet))

    details = data_wavelet[1:]
    print(len(details))
    


### Filtro wavelet para usar

In [2]:
def filtro_wavelet(signal_data, wavelet='db4', nivel=4):
    coeffs = pywt.wavedec(signal_data, wavelet, level=nivel)
    umbral = np.median(np.abs(coeffs[-1])) / 0.6745
    coeffs = [pywt.threshold(c, umbral, mode='soft') for c in coeffs]
    return pywt.waverec(coeffs,wavelet)

4. Escoger 10 señales al azar y aplicar un flujo de procesamiento

In [7]:
def cargar_datos(selected_files):
    # Diccionario para almacenar cada DataFrame y su columna 'II'
    data_frames = {}
    datos_II = {}
    
    # Iterar sobre los archivos en la lista
    for i, file in enumerate(selected_files):
        # Leer cada archivo y guardarlo en el diccionario con un nombre único
        df = pd.read_csv(f"./datos/ECGDataDenoised/{file}.csv", delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
        data_frames[f"df{i+1}"] = df
        # datos_II[f"datos_f{i+1}"] = df["II"]
        datos_II[f"{file}"] = df["II"]
    
    return data_frames, datos_II


In [8]:
# Separar nombres de archivo en listas según el ritmo 'SB' y 'AFIB'
sb_files = datos_diagnostics[datos_diagnostics['Rhythm'] == 'SB']['FileName'].tolist()
afib_files = datos_diagnostics[datos_diagnostics['Rhythm'] == 'AFIB']['FileName'].tolist()

In [9]:
# Número total de archivos a seleccionar
n_seleccion = 10

# Calcular la mitad para cada ritmo (o una distribución casi mitad y mitad si n_seleccion es impar)
n_sb = n_seleccion // 2
n_afib = n_seleccion - n_sb

# Seleccionar aleatoriamente el número calculado de archivos de cada tipo
selected_sb_files = random.sample(sb_files, min(n_sb, len(sb_files)))
selected_afib_files = random.sample(afib_files, min(n_afib, len(afib_files)))

# Combinar las selecciones de 'SB' y 'AFIB' en la lista final
selected_files = selected_sb_files + selected_afib_files

# Mezclar los archivos seleccionados para no tener un orden específico de 'SB' o 'AFIB'
random.shuffle(selected_files)
# Filtrar el DataFrame para obtener solo las filas de los archivos en selected_files
selected_data = datos_diagnostics[datos_diagnostics['FileName'].isin(selected_files)]

# Crear un diccionario con 'FileName' como clave y 'Rhythm' como valor para fácil acceso
file_rhythm_dict = dict(zip(selected_data['FileName'], selected_data['Rhythm']))

In [10]:
data_frames, datos_II = cargar_datos(selected_files)

In [11]:
datos_II.keys()

dict_keys(['MUSE_20180118_130405_09000', 'MUSE_20180112_120649_89000', 'MUSE_20180118_173302_02000', 'MUSE_20180114_132312_14000', 'MUSE_20180115_115742_25000', 'MUSE_20180118_132137_58000', 'MUSE_20180116_133618_66000', 'MUSE_20180114_121206_37000', 'MUSE_20180112_073749_76000', 'MUSE_20180113_133639_30000'])

In [12]:
#Aplicar Wavelet 
for archivo in selected_files:
    # Leer el archivo CSV
    df = pd.read_csv(f"../Proyecto_3/datos/ECGDataDenoised/{archivo}.csv", delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
    datos_II=df["II"]
    filtro_wavelet(datos_II)

### Flujos de trabajo

### Flujo 1
1. Filtro pasa-altas usando filtro IIR a 0.5 Hz. Justificar la elección de parámetros y si se usa FIR o IIR
2. Filtro wavelet modificado del punto 3
3. Filtrado pasabajas 50 Hz. Justificar la elección de parámetros y si se usa FIR o IIR

In [14]:
# Flujo 1: Filtro pasa-altas, Filtro wavelet, Filtro pasabajas
def flujo_1(data,fs = 1000 ):
    # Paso 1: Filtro pasa-altas IIR a 0.5 Hz
    cutoff = 0.5  # Frecuencia de corte en Hz
    sos = signal.butter(4, cutoff, btype='highpass', fs=fs, output='sos')
    data_highpass = signal.sosfilt(sos, data)
    
    # Paso 2: Filtro wavelet

    wavelet='db4'
    coeffs = pywt.wavedec(data_highpass, wavelet, level=4)
    umbral = np.median(np.abs(coeffs[-1])) / 0.6745
    coeffs = [pywt.threshold(c, umbral, mode='soft') for c in coeffs]
    data_denoised = pywt.waverec(coeffs, wavelet)
 
    
    # Paso 3: Filtro pasabajas IIR a 50 Hz
    cutoff = 50  # Frecuencia de corte en Hz
    sos = signal.butter(4, cutoff, btype='lowpass', fs=fs, output='sos')
    data_lowpass = signal.sosfilt(sos, data_denoised)
    
    return data_lowpass



# def flujo_1(data,fs = 1000 ):
#     # Paso 1: Filtro pasa-altas IIR a 0.5 Hz
#     cutoff = 0.5  # Frecuencia de corte en Hz
#     sos = signal.butter(4, cutoff, btype='highpass', fs=fs, output='sos')
#     data_highpass = signal.sosfilt(sos, data)
    
#     # Paso 2: Filtro wavelet
#     data_wavelet = pywt.wavedec(data_highpass, 'db6', level=8)
#     details = data_wavelet[1:]
#     thr, s, denoised_details = wthresh(details)  # Usa la función wthresh definida previamente
#     data_wavelet_denoised = [data_wavelet[0]] + denoised_details
#     data_denoised = pywt.waverec(data_wavelet_denoised, 'db6')
    
#     # Paso 3: Filtro pasabajas IIR a 50 Hz
#     cutoff = 50  # Frecuencia de corte en Hz
#     sos = signal.butter(4, cutoff, btype='lowpass', fs=fs, output='sos')
#     data_lowpass = signal.sosfilt(sos, data_denoised)
    
#     return data_lowpass

### Flujo 2
1. Detrend
2. Filtro wavelet modificado del punto 3
3. Filtrado pasabajas 50 Hz. Justificar la elección de parámetros y si se usa FIR o IIR

In [16]:
# Flujo 2: Detrend, Filtro wavelet, Filtro pasabajas
def flujo_2(data,fs = 1000 ):
    # Paso 1: Detrend
    data_detrended = signal.detrend(data)
    
    # Paso 2: Filtro wavelet
    wavelet='db4'
    coeffs = pywt.wavedec(data_detrended, wavelet, level=4)
    umbral = np.median(np.abs(coeffs[-1])) / 0.6745
    coeffs = [pywt.threshold(c, umbral, mode='soft') for c in coeffs]
    data_denoised = pywt.waverec(coeffs, wavelet)
    
    # Paso 3: Filtro pasabajas IIR a 50 Hz
    cutoff = 50  # Frecuencia de corte en Hz
    sos = signal.butter(4, cutoff, btype='lowpass', fs=fs, output='sos')
    data_lowpass = signal.sosfilt(sos, data_denoised)
    
    return data_lowpass



# # Flujo 2: Detrend, Filtro wavelet, Filtro pasabajas
# def flujo_2(data,fs = 1000 ):
#     # Paso 1: Detrend
#     data_detrended = signal.detrend(data)
    
#     # Paso 2: Filtro wavelet
#     data_wavelet = pywt.wavedec(data_detrended, 'db6', level=8)
#     details = data_wavelet[1:]
#     thr, s, denoised_details = wthresh(details)  # Usa la función wthresh definida previamente
#     data_wavelet_denoised = [data_wavelet[0]] + denoised_details
#     data_denoised = pywt.waverec(data_wavelet_denoised, 'db6')
    
#     # Paso 3: Filtro pasabajas IIR a 50 Hz
#     cutoff = 50  # Frecuencia de corte en Hz
#     sos = signal.butter(4, cutoff, btype='lowpass', fs=fs, output='sos')
#     data_lowpass = signal.sosfilt(sos, data_denoised)
    
#     return data_lowpass


### Flujo 3
1. Filtro pasa-altas usando filtro IIR a 0.5 Hz. Justificar la elección de parámetros y si se usa FIR o IIR
2. Filtrado pasabajas 50 Hz. Justificar la elección de parámetros y si se usa FIR o IIR

In [23]:
# Flujo 3: Filtro pasa-altas, Filtro pasabajas
def flujo_3(data,fs = 1000 ):
    # Paso 1: Filtro pasa-altas IIR a 0.5 Hz
    cutoff = 0.5  # Frecuencia de corte en Hz
    sos = signal.butter(4, cutoff, btype='highpass', fs=fs, output='sos')
    data_highpass = signal.sosfilt(sos, data)
    

    # Paso 2: Filtro pasabajas IIR a 50 Hz
    cutoff = 50  # Frecuencia de corte en Hz
    sos = signal.butter(4, cutoff, btype='lowpass', fs=fs, output='sos')
    data_lowpass = signal.sosfilt(sos, data_highpass)
    
    return data_lowpass



# # Flujo 3: Filtro pasa-altas, Filtro pasabajas
# def flujo_3(data,fs = 1000):
#     # Paso 1: Filtro pasa-altas IIR a 0.5 Hz
#     cutoff = 0.5  # Frecuencia de corte en Hz
#     sos = signal.butter(4, cutoff, btype='highpass', fs=fs, output='sos')
#     data_highpass = signal.sosfilt(sos, data)
    
#     # Paso 2: Filtro pasabajas IIR a 50 Hz
#     cutoff = 50  # Frecuencia de corte en Hz
#     sos = signal.butter(4, cutoff, btype='lowpass', fs=fs, output='sos')
#     data_lowpass = signal.sosfilt(sos, data_highpass)
    
#     return data_lowpass


In [24]:
#Ej aplicación de flujos
# flujo_1(datos_II["MUSE_20180118_130405_09000"],fs)

#Aplicar Wavelet 
for archivo in selected_files[:2]:
    # Leer el archivo CSV
    df = pd.read_csv(f"../Proyecto_3/datos/ECGDataDenoised/{archivo}.csv", delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
    datos_II=df["II"]
    flujo_3(datos_II)

Describir los resultados obtenidos y decidir si el resto del procesamiento se hace con el flujo 1, el flujo 2 o el flujo 3 (20%)

## Ecuaciones proyecto 2

In [ ]:
def Grafica(tiempo,data,datos_process,nombre="Procesado"):
    
    # Crear una figura con dos subplots en una fila
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15, 4))
    # Graficar en el primer subplot
    ax1.plot(tiempo,data, color="k")
    ax1.set_title('Original')
    ax1.grid(True)
    # Graficar en el segundo subplot
    ax2.plot(tiempo,datos_process, color="r", linestyle='-')
    ax2.set_title(nombre)
    ax2.grid(True)
    plt.show()

In [27]:
#Ecuación 1
def detrend_n_deviation(data,tiempo):
    datos_detrend=signal.detrend(data)
    datos_deviation=np.mean((data-datos_detrend)**2)
    return datos_detrend, datos_deviation

In [28]:
# Ecuación 2
def compress(signal):
    xmin, xmax = np.min(signal), np.max(signal)
    if xmin == xmax:
        return signal  # Evitar dividir por cero si no hay rango
    compressed_signal = (signal - xmin) / (xmax - xmin)
    return np.array(compressed_signal)

In [29]:
#Ecuaciones 3 y 4
def bispectrum_bicoherence(signal, fs, segment):
    # Dividir la señal en segmentos de longitud nfft
    segmentos = [signal[i:i + segment] for i in range(0, len(signal) - segment + 1, segment)]
    segmentos = np.array(segmentos)
    num_segmentos = segmentos.shape[0]

    # Aplicar la Transformada de Fourier a cada segmento
    fft_segmentos = np.fft.fft(segmentos, n=segment, axis=1)[:, :segment // 2]

    # Inicializar matrices para el bispectro y la bicoherencia
    bispectro = np.zeros((segment // 2, segment // 2), dtype=complex)
    acumulado = np.zeros((segment // 2, segment // 2), dtype=complex)

    # Calcular el bispectro y el valor acumulado para la bicoherencia
    for k in range(num_segmentos):
        S = fft_segmentos[k, :]
        for f1 in range(segment // 2):
            for f2 in range(f1, segment // 2 - f1):
                f3 = f1 + f2
                if f3 < segment // 2:  # Evitar exceder los límites del espectro
                    producto = S[f1] * S[f2] * np.conj(S[f3])
                    bispectro[f1, f2] += producto
                    acumulado[f1, f2] += np.abs(producto)

    # Normalizar el bispectro para obtener la bicoherencia
    bicoherencia = np.abs(bispectro) / acumulado

    # Aplicar simetría a la matriz de bicoherencia
    return bispectro, bicoherencia


def aplicar_simetria(matriz):
    matriz_simetrica = np.zeros_like(matriz)
    for f1 in range(matriz.shape[0]):
        for f2 in range(f1, matriz.shape[1] - f1):
            matriz_simetrica[f1, f2] = matriz[f1, f2]
    return matriz_simetrica


In [30]:
# Para cada señal extraer la frecuencia que contiene la máxima potencia y Gráfiquela 
def obtener_frecuencia_maxima_potencia(df, fs=1000):
    """
    Calcula la frecuencia con la máxima potencia en el espectro de una señal utilizando Welch.

    Parámetros:
    df : pd.DataFrame - DataFrame que contiene la columna 'signal' con la señal.
    fs : int - Frecuencia de muestreo de la señal (en Hz). Ajustar según los datos reales.

    Retorna:
    frecuencia_max : float - La frecuencia con la máxima potencia.
    """
    # Asegurarse de que la columna 'signal' existe en el dataframe
    # if 'II' not in df.columns:
    #     raise ValueError("El DataFrame debe contener una columna llamada 'signal'.")

    # Calcular la densidad espectral de potencia utilizando Welch
    frecuencias, potencia = welch(df, fs=fs)

    # Encontrar la frecuencia con la máxima potencia
    indice_max_potencia = potencia.argmax()
    frecuencia_max = frecuencias[indice_max_potencia]

    return frecuencia_max

# # Función para calcular la frecuencia de máxima potencia
# def max_power_frequency(signal, fs):
#     # Calcular la transformada de Fourier del señal
#     fft_vals = np.fft.fft(signal)
#     # Mantener solo la mitad de los valores (espectro positivo)
#     fft_magnitude = np.abs(fft_vals[:len(fft_vals)//2])
#     # Calcular las frecuencias correspondientes
#     frequencies = np.fft.fftfreq(len(signal), d=1/fs)[:len(fft_vals)//2]
#     # Calcular la potencia en el dominio de la frecuencia
#     power_spectrum = fft_magnitude**2
#     # Encontrar el índice de la frecuencia con máxima potencia
#     max_power_idx = np.argmax(power_spectrum)
#     # Obtener la frecuencia de máxima potencia y el valor de la potencia máxima
#     max_frequency = round(frequencies[max_power_idx], 5)
#     max_power_value = power_spectrum[max_power_idx]
#     # Graficar la magnitud de la transformada de Fourier y marcar la frecuencia de máxima potencia
#     plt.figure()
#     plt.plot(frequencies, fft_magnitude)
#     plt.axvline(x=max_frequency, color='k', linestyle='--', label=f'Máxima Frecuencia: {max_frequency} Hz')
#     plt.legend()

#     return max_frequency, max_power_value


5. Para cada señal sumar el número de frecuencias que son significativas para bicoherencia (cuales tienen un valor superior a ![image](https://github.com/user-attachments/assets/636a4170-c3f7-4789-9100-f2a0ce0033c3) ) (20%)

In [17]:
# Función para sumar las frecuencias significativas en bicoherencia
def significant_frequencies_bicoherence(bicoherence, N):
    threshold = np.sqrt(9.2 / (2 * N))
    significant_frequencies = np.sum(bicoherence > threshold)
    return significant_frequencies


6. Crear una rutina que aplique sobre todos los archivos de la base de datos las rutina 3 al 5 y almacene los resultados en un dataframe donde se pueda registro, tipo de patología y el frecuencia de máxima potencia (fMP) y suma de frecuencias significativas en bicoherencia (sFSB):

| Registro | Estado | fMP | sFSB  |
| --- | --- | --- | --- |
| --- | --- | --- | --- |
| --- | --- | --- | --- |


In [31]:
def signal_process(datos,tiempo,fs):
    print("Aplicando detrend")
    data_detrended=detrend_n_deviation(datos,tiempo)
    print(f'La desviación es {data_detrended[1]}')
    # Grafica(tiempo,datos,data_detrended[0],nombre="Detrended")

    print("Aplicando compress")
    data_compressed=compress(data_detrended[0])
    # Grafica(tiempo,data_detrended[0],data_compressed,"Compressed")

    print("Aplicando biespectro y bicoherencia")
    bisp_Bic=bispectrum_bicoherence(data_compressed,fs,1000)
    # Grafica(datos[:len(bisp_Bic[0])],data_compressed[:len(bisp_Bic[0])],bisp_Bic[0],"bispectrum and bicoherence")
    # Grafica(tiempo[:len(bisp_Bic[0])],data_compressed[:len(bisp_Bic[0])],bisp_Bic[0],"bispectrum")
    # Calcular el bispectro

    bisp_Bic = bispectrum_bicoherence(datos_II, fs, 1000)
    bisp_Bic= aplicar_simetria(bisp_Bic[1])
    # Graficar el bicoherence
    # plt.figure(figsize=(8, 5))
    # plt.imshow(np.abs(bisp_Bic), extent=(0, 20, 0, 10), aspect='auto', origin='lower', cmap='viridis')
    # plt.colorbar(label='Magnitud de bicoherencia')
    # plt.xlabel('Frecuencia f1 (Hz)')
    # plt.ylabel('Frecuencia f2 (Hz)')
    # plt.title('Bicoherencia Simétrica')
    # plt.show()

    print("calculando la máxima potencia")
    max_p=obtener_frecuencia_maxima_potencia(datos,fs)
    print(f'La máxima potencia es {[max_p]}')
    # plt.figure(figsize=(10, 5))
    # # plt.semilogy(max_p[1], max_p[2], label='Densidad Espectral de Potencia')
    # plt.axvline(max_p[0], color='r', linestyle='--', label=f'Máxima Frecuencia: {max_p[0]:.2f} Hz')

    # plt.plot(max_p[1], max_p[2])
    # plt.xlim([0,20])

    
    # plt.title('Densidad Espectral de Potencia')
    # plt.xlabel('Frecuencia [Hz]')
    # plt.ylabel('Densidad Espectral [V^2/Hz]')
    # plt.legend()
    # plt.grid()
    # plt.show()

    # print("calculando Frecuencias significativas")
    # sig_f=significant_frequencies_bicoherence(bisp_Bic[1],len(bisp_Bic[1]))
    # print(f'Frecuencias significativas para bicoherencia: {sig_f} Hz \n')
    
    # return data_detrended, data_compressed, bisp_Bic, max_p, sig_f
    return data_detrended, data_compressed, bisp_Bic, max_p

### Aplicar procesado a señal de derivada II por metodo proyecto 2

7. Para las señales de análisis del punto 2 aplicar las rutinas del punto 3 al 5 y hacer un informe que permita evidenciar las diferencias entre las señales SB y AFIB utilizando las herramientas de estadística descriptiva (5%)

In [33]:
# Crear un DataFrame vacío para almacenar los resultados
# df_resultados = pd.DataFrame(columns=["Registro", "Estado", "fMP", "sFSB"])
df_resultados = pd.DataFrame(columns=["Registro", "Estado", "fMP"])
# ruta_carpeta="../Proyecto_3/datos/"
# rut=".csv"

for archivo in selected_files:
    # Leer el archivo CSV
    df = pd.read_csv(f"../Proyecto_3/datos/ECGDataDenoised/{archivo}.csv", delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
    datos_II=df["II"]
    tiempo = np.arange(0, datos_II.shape[0]/fs,1/fs)

# Obtener el nombre del sujeto (sin extensión .csv)
    nombre_sujeto = archivo[:-4] #Registro

# Procesar los archivos
    resultados_proceso=signal_process(datos_II,tiempo,fs)

    estado = file_rhythm_dict[archivo] #Estado

    F_max=resultados_proceso[3] #fMP
    
    # F_sig=resultados_proceso[4] #sFSB

    # df_resultados = pd.concat([df_resultados,
    #                     pd.DataFrame({'Registro': nombre_sujeto,
    #                                     'Estado': estado,
    #                                     'fMP': F_max,
    #                                     'sFSB': F_sig},
    #                                 index=[0])
    #                     ],
    #                     ignore_index=True)
    
    df_resultados = pd.concat([df_resultados,
                    pd.DataFrame({'Registro': nombre_sujeto,
                                    'Estado': estado,
                                    'fMP': F_max},
                                index=[0])
                    ],
                    ignore_index=True)

Aplicando detrend
La desviación es 27.81953570693658
Aplicando compress
Aplicando biespectro y bicoherencia


C:\Users\crist\AppData\Local\Temp\ipykernel_16448\3190863204.py:27: RuntimeWarning: invalid value encountered in divide
  bicoherencia = np.abs(bispectro) / acumulado


calculando la máxima potencia
La máxima potencia es [np.float64(7.8125)]
Aplicando detrend
La desviación es 2330.9675138779708
Aplicando compress
Aplicando biespectro y bicoherencia


C:\Users\crist\AppData\Local\Temp\ipykernel_16448\1847125809.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados = pd.concat([df_resultados,
C:\Users\crist\AppData\Local\Temp\ipykernel_16448\3190863204.py:27: RuntimeWarning: invalid value encountered in divide
  bicoherencia = np.abs(bispectro) / acumulado


calculando la máxima potencia
La máxima potencia es [np.float64(3.90625)]
Aplicando detrend
La desviación es 3299.240163107669
Aplicando compress
Aplicando biespectro y bicoherencia


...

calculando la máxima potencia
La máxima potencia es [np.float64(3.90625)]
Aplicando detrend
La desviación es 127.85626965481939
Aplicando compress
Aplicando biespectro y bicoherencia


C:\Users\crist\AppData\Local\Temp\ipykernel_16448\3190863204.py:27: RuntimeWarning: invalid value encountered in divide
  bicoherencia = np.abs(bispectro) / acumulado


calculando la máxima potencia
La máxima potencia es [np.float64(1.953125)]
Aplicando detrend
La desviación es 1349.1182037329459
Aplicando compress
Aplicando biespectro y bicoherencia


C:\Users\crist\AppData\Local\Temp\ipykernel_16448\3190863204.py:27: RuntimeWarning: invalid value encountered in divide
  bicoherencia = np.abs(bispectro) / acumulado


calculando la máxima potencia
La máxima potencia es [np.float64(1.953125)]


In [34]:
df_resultados

,Registro,Estado,fMP
0,MUSE_20180118_130405_0,AFIB,7.812500
1,MUSE_20180112_120649_8,AFIB,3.906250
2,MUSE_20180118_173302_0,AFIB,1.953125
3,MUSE_20180114_132312_1,SB,7.812500
4,MUSE_20180115_115742_2,AFIB,3.906250
5,MUSE_20180118_132137_5,SB,1.953125
6,MUSE_20180116_133618_6,SB,3.906250
7,MUSE_20180114_121206_3,SB,3.906250
8,MUSE_20180112_073749_7,AFIB,1.953125
9,MUSE_20180113_133639_3,SB,1.953125


In [ ]:
# df_resultados.to_excel("Dataframe_P2.xlsx", index=False)

# Para todos los datos

In [ ]:
all_files = datos_diagnostics['FileName'].tolist()

In [ ]:
# Crear un DataFrame vacío para almacenar los resultados
# df_resultados = pd.DataFrame(columns=["Registro", "Estado", "fMP", "sFSB"])
df_resultados = pd.DataFrame(columns=["Registro", "Estado", "fMP"])
# ruta_carpeta="../Proyecto_3/datos/"
# rut=".csv"

for archivo in all_files:
    # Leer el archivo CSV
    df = pd.read_csv(f"../Proyecto_3/datos/ECGDataDenoised/{archivo}.csv", delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
    datos_II=df["II"]
    tiempo = np.arange(0, datos_II.shape[0]/fs,1/fs)

# Obtener el nombre del sujeto (sin extensión .csv)
    nombre_sujeto = archivo[:-4] #Registro

# Procesar los archivos
    resultados_proceso=signal_process(datos_II,tiempo,fs)

    estado = file_rhythm_dict[archivo] #Estado

    F_max=resultados_proceso[3] #fMP
    
    # F_sig=resultados_proceso[4] #sFSB

    # df_resultados = pd.concat([df_resultados,
    #                     pd.DataFrame({'Registro': nombre_sujeto,
    #                                     'Estado': estado,
    #                                     'fMP': F_max,
    #                                     'sFSB': F_sig},
    #                                 index=[0])
    #                     ],
    #                     ignore_index=True)
    
    df_resultados_all = pd.concat([df_resultados,
                    pd.DataFrame({'Registro': nombre_sujeto,
                                    'Estado': estado,
                                    'fMP': F_max},
                                index=[0])
                    ],
                    ignore_index=True)

In [ ]:
df_resultados_all

In [ ]:
df_resultados.to_excel("Dataframe_P2.xlsx", index=False)

array([  0.17636665,   1.30513762,   4.65246662, ..., -41.26171485,
       -39.07229233, -36.15797145])

8. Indicar si hay o no diferencias estadísticas entre las características espectrales del punto 4 y 5 para las dos poblaciones de estudio (5%)


Prueba de Normalidad

Hipótesis nula (Ho): Los datos recolectados del ECG no distribuyen normal

Hipotesis alternativa (H1): Los datos distribuyen de manera normal

In [ ]:
#Prueba de Normalidad
alpha=0.05 #Valor de significancia de 5%
cols=df_resultados.columns[2:]
for i in cols:
    [statistic,pvalue]=stats.normaltest(df_resultados[i])
    if pvalue > alpha:
        print(f'la columna {i} distribuye normal')
    else:
        print(f'la columna {i} no distribuye normal')

Prueba Homocedasticidad  
Hipótesis nula (Ho): Los varianza de los datos recolectados del ECG no son significativamente diferentes

Hipotesis alternativa (H1): La varianza de los datos son significativamente diferentes

In [ ]:
#Prueba Homocedasticidad
# [statistic,pvalue]=stats.levene(ciclos[2],ciclos[3],ciclos[13],ciclos[14])
[statistic,pvalue]=stats.levene(df_resultados["fMP"],df_resultados["sFSB"])
print(f'Valor p: {pvalue} \n')
if pvalue > alpha:
    print('Cumple supuesto de homocedasticidad') #Varianzas no son significativamente diferentes
else:
    print('No cumple supuesto de homocedasticidad')

Prueba Mann-Whitney

Hipótesis nula (Ho): No hay diferencia entre la frecuencia significativa en bicoherencia entre pacientes con bradicardia sinusal (SB) y fibrilación auricular (AFIB)
Hipotesis alternativa (H1): Si hay diferencia de la frecuencia significativa en bicoherencia entre pacientes con bradicardia sinusal (SB) y fibrilación auricular (AFIB)

In [ ]:
#Prueba Mann-Whitney
df_resultados["sFSB"]= pd.to_numeric(df_resultados["sFSB"], errors='coerce')
grupo1 = df_resultados["fMP"]
grupo2 = df_resultados["sFSB"]
[statistic,pvalue]=stats.mannwhitneyu(grupo1,grupo2)
if pvalue < alpha:
    print('Existe diferencia significativa entre fMP y sFSB ')
else:
    print('No existe diferencia significativa entre fMP y sFSB')
print(f'Valor p: {pvalue} \n')

